In [1]:
%matplotlib inline
%env KERAS_BACKEND=tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from ipywidgets import interact, IntSlider, Button, fixed
from ipywidgets import interact_manual
# import sympy as sym
# from sympy import S,N
# import locale
# from sklearn.linear_model import LinearRegression
from keras.utils import np_utils #one hot encoding
# from keras.models import Sequential #神經網路的框架keras.models
from keras.layers import Dense, Activation
from keras.layers import Flatten # 將資料展平
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta
from keras.models import model_from_json
from keras.datasets import mnist # mnist內 包含手寫測試資料

#for Functional API
from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate, add
#for Functional API transfer learning

from keras.layers.core import Lambda
from keras import backend as K

# from keras.preprocessing.image import ImageDataGenerator
# from keras.layers import Dropout

import matplotlib.image as mpimg #import image file
# from sklearn.model_selection import train_test_split

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


In [2]:
(x_train0, y_train0), (x_test0, y_test0) = mnist.load_data()
print("There are %d training data with size %d x %d" %x_train0.shape)
print("There are %d testing  data with size %d x %d" %x_test0.shape)
# y_test0.shape

x_train = x_train0.reshape(x_train0.shape[0], 28, 28, 1)
x_test = x_test0.reshape(x_test0.shape[0], 28, 28, 1)
x_train -= x_train.min()
x_train = x_train/x_train.max()
y_train = np_utils.to_categorical(y_train0, 10)
y_test = np_utils.to_categorical(y_test0, 10)

print("x_train.max = ", x_train.max(), "\nx_train.min = ", x_train.min(), "\ny_test[0] = ", y_test[0])

There are 60000 training data with size 28 x 28
There are 10000 testing  data with size 28 x 28
x_train.max =  1.0 
x_train.min =  0.0 
y_test[0] =  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [3]:
x = Input(shape = (28, 28, 1))
print("x = ", x)
f_1 = Conv2D(4,(4,4), padding = 'same', activation = 'relu') # Number of filters: 4
# f_1.trainable = False
pool1 = MaxPool2D(pool_size = (2, 2))
f_2 = Conv2D(8,(4,4), padding = 'same', activation = 'relu') # Number of filters: 8
pool2 = MaxPool2D(pool_size = (2, 2))
f_3 = Conv2D(16,(4,4), padding = 'same', activation = 'relu') # Number of filters: 16
pool3 = MaxPool2D(pool_size = (2, 2))
f_4 = Dense(80, activation='relu')
f_5 = Dense(10, activation='softmax')
print("f_1 = ", f_1) # keras.layers.core.Dense object

x =  Tensor("input_1:0", shape=(?, 28, 28, 1), dtype=float32)
f_1 =  <keras.layers.convolutional.Conv2D object at 0x0000027C29C1F7F0>


In [4]:
h_1 = pool1(f_1(x))
h_2 = pool2(f_2(h_1))
h_3 = pool3(f_3(h_2))
h_4 = Flatten() (h_3)
h_5 = f_4(h_4)
y = f_5(h_5)
print("h_1 = ", h_1)

Instructions for updating:
Colocations handled automatically by placer.
h_1 =  Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 14, 14, 4), dtype=float32)


In [5]:
model = Model(x, y)

model.summary()
# Trainable 的問題可以在定義f_i的時候加上 " f_i.trainable = False "

model.compile(loss='categorical_crossentropy', optimizer=Adadelta(), metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 4)         68        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 4)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 8)         520       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 16)          2064      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 3, 3, 16)          0         
__________

In [6]:
model.fit(x_train, y_train, batch_size=100, epochs=3)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
60000/60000 [==============================] - 26s 431us/step - loss: 0.3120 - acc: 0.8994
Epoch 2/3
60000/60000 [==============================] - 18s 301us/step - loss: 0.0869 - acc: 0.9725
Epoch 3/3
60000/60000 [==============================] - 26s 441us/step - loss: 0.0664 - acc: 0.9790


In [7]:
score = model.evaluate(x_test, y_test, batch_size=10000)
print("Loss: %f" %score[0])
print("準確率: %f" %(score[1]*100))

10000/10000 [==============================] - 2s 216us/step
Loss: 0.294454
準確率: 98.159999


In [8]:
# layer0 = f_1
def show_filter_weight(index, layer):
# def show_filter_weight(index):
    filterweight = layer.get_weights()[0].transpose(3, 2, 0, 1)
    filterweight = filterweight.reshape(-1,4,4)
#     print(filterweight.shape)
    ele = filterweight[index]
#     print(ele.shape)
    normal = ele-ele.reshape(-1).min()/ele.reshape(-1).max()
    plt.imshow(normal.reshape(4,4), cmap='Greys')

def show_interact(layer):
    interact(show_filter_weight, 
         index = (0, layer.get_weights()[0].transpose(3, 2, 0, 1).reshape(-1,4,4).shape[0]-1), 
         layer = fixed(layer)
         )
interact(show_interact, layer = {1:f_1, 2:f_2, 3:f_3});

# interact(show_filter_weight, 
#     index = (0, layer0.get_weights()[0].transpose(3, 2, 0, 1).reshape(-1,4,4).shape[0]-1)
#     );

interactive(children=(Dropdown(description='layer', options={1: <keras.layers.convolutional.Conv2D object at 0…

In [9]:
# model_json = model.to_json() # 將神經網路變成json格式
# open('json_h5/Functional_API.json','w').write(model_json)
# model.save_weights('json_h5/Functional_API.h5') # 將整個model存起來

## 讀入

In [10]:
# new_model = model_from_json(open('json_h5/Functional_API.json').read())
# new_model.load_weights('json_h5/Functional_API.h5')
# new_model.compile(loss='categorical_crossentropy', optimizer=Adadelta(), metrics=['accuracy'])

In [11]:
# score = new_model.evaluate(x_test, y_test, batch_size=10000)
# print("Loss: %f" %score[0])
# print("準確率: %f" %(score[1]*100))